In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
from fastai import *        # Quick accesss to most common functionality
# from fastai.text import *   # Quick accesss to NLP functionality
import html
from pathlib import Path

In [3]:
import torch.nn as nn

In [4]:
from fastai.text.models.transformer import *
from fastai.text.models.transformer import init_transformer
from fastai.text.models.awd_lstm import RNNDropout, LinearDecoder
from fastai.text.learner import LanguageLearner

In [5]:
PATH=Path.home()/'data/wikitext-103-raw'

In [6]:
from fastai.basic_data import *
from fastai.torch_core import *
from fastai.layers import *

In [7]:
bptt = 512
data = load_data(PATH, bs=16, bptt=bptt)

In [8]:
MASK = 'xxmask'
vocab = data.vocab
vocab.itos.append(MASK)

In [9]:
word_range = (0, len(data.vocab.itos))
from fastai.text.transform import *
pad_idx = data.vocab.stoi[PAD]
mask_idx = data.vocab.stoi[MASK]
def bert_tfm(b, word_range=word_range, pad_idx=pad_idx, 
             mask_idx=mask_idx, p=0.2):
    # p = replacement probability
    x_lm,y_lm = b
    x_msk,y_msk = x_lm.clone(),x_lm.clone() # x, x
    rand = torch.rand(x_msk.shape, device=x_lm.device)
    y_msk[rand > p] = pad_idx
    x_msk[rand <= (p*.8)] = mask_idx # 80% = mask
    wrong_word = (rand > (p*.8)) & (rand <= (p*.9)) # 10% = wrong word
    x_msk[wrong_word] = torch.randint(*word_range, [wrong_word.sum().item()], device=x_lm.device)
    return x_msk, y_msk

In [10]:
data.train_dl.tfms = [bert_tfm]
data.valid_dl.tfms = [bert_tfm]

In [11]:

def acc_ignore_pad(input:Tensor, targ:Tensor, pad_idx=pad_idx)->Rank0Tensor:
    n = targ.shape[0]
    input = input.argmax(dim=-1).view(n,-1)
    targ = targ.view(n,-1)
    mask = targ != pad_idx
    return (input[mask]==targ[mask]).float().mean()

def bert_acc(input:Tensor, b_t:Tensor)->Rank0Tensor:
    return acc_ignore_pad(input, b_t)

In [12]:

class Decoder(nn.Module):
    "To go on top of a RNNCore module and create a Language Model."
    initrange=0.1

    def __init__(self, n_out:int, n_hid:int, output_p:float=0.0, tie_encoder:nn.Module=None, bias:bool=True):
        super().__init__()
        self.decoder = nn.Linear(n_hid, n_out, bias=bias)
        self.decoder.weight.data.uniform_(-self.initrange, self.initrange)
        self.output_dp = RNNDropout(output_p)
        if bias: self.decoder.bias.data.zero_()
        if tie_encoder: self.decoder.weight = tie_encoder.weight

    def forward(self, input:Tuple[Tensor,Tensor])->Tuple[Tensor,Tensor,Tensor]:
        output = self.output_dp(input)
        decoded = self.decoder(output)
        return decoded

In [13]:
class TransformerBlock(nn.Module):
    "Basic block of a Transformer model."
    #Can't use Sequential directly cause more than one input...
    def __init__(self, d_attn:int, n_heads:int, d_head:int, d_out:int=None, bias:bool=True, ff_p=0.0):
        super().__init__()
        if d_out is None: d_out = d_attn
        d_inner = d_out*4
        self.mhra = MultiHeadRelativeAttention(n_heads, d_attn, d_head, bias=bias)
#         self.mhra = MultiHeadAttention(n_heads, d_attn, d_head, bias=bias)

        self.ln = nn.Sequential(*[
            nn.Linear(d_out, d_inner), 
            GeLU(),
            nn.Linear(d_inner, d_out),
            nn.Dropout(ff_p), 
        ])
        self.norm = nn.LayerNorm(d_out)
        
        self.pos_enc = PositionalEncoding(d_attn)
        self.x_p = None
        
    def attn(self, x:Tensor, **kwargs):
#         return self.mhra(x)
        if self.x_p is None:
            pos = torch.arange(x.shape[1]-1, -1, -1, device=x.device, dtype=x.dtype)
            self.x_p = self.pos_enc(pos)
        u, v = 0, 0
        return self.mhra(x, r=self.x_p, u=u, v=v)
        
    def forward(self, x:Tensor, **kwargs): 
        x_attn = self.attn(x, **kwargs)
        res = self.ln(x_attn) + x_attn
        return self.norm(res)

In [14]:
class DownsampleLayer(TransformerBlock):
    "Basic block of a Transformer model."
    #Can't use Sequential directly cause more than one input...
    def __init__(self, d_model:int, n_heads:int, d_head:int):
        d_out = d_model * 2
        super().__init__(d_out, n_heads, d_head)
        self.downblock = nn.Conv1d(d_model, d_out, (2), stride=2)
            
    def forward(self, x:Tensor, **kwargs):
        x_d = self.downblock(x.permute(0, 2, 1)).permute(0, 2, 1) # bptt x emb x bptt
        
        x_attn = self.attn(x_d, **kwargs)
        
        x1 = self.ln(x_attn)
        
        return self.norm(x_attn + x1)

In [15]:
class UpsampleLayer(TransformerBlock):
    "Basic block of a Transformer model."
    #Can't use Sequential directly cause more than one input...
    def __init__(self, d_model:int, n_heads:int, d_head:int):
        d_out = d_model // 2
        super().__init__(d_out, n_heads, d_head)
        self.upblock = nn.ConvTranspose1d(d_model, d_out, (2), stride=2)
    
    def forward(self, x:Tensor, x_skip:Tensor, **kwargs):
        x_u = self.upblock(x.permute(0, 2, 1)).permute(0, 2, 1) # bptt x emb x bptt
        
        x_attn = self.attn(x_u, **kwargs)
        
        x1 = self.ln(x_attn) + x_attn
        
        return self.norm(x1 + x_skip)

## Transformer

In [16]:
class Embedder(nn.Module):
    "Embedding + positional encoding + dropout"
    def __init__(self, vocab_sz:int, d_model:int, embed_p:float=0.):
        super().__init__()
        self.embed = nn.Embedding(vocab_sz, d_model)
        # See https://arxiv.org/abs/1711.09160
        with torch.no_grad(): trunc_normal_(self.embed.weight, std=0.01)
        self.drop = nn.Dropout(embed_p)
    
    def forward(self, inp, pos_forward=False):
        emb = self.drop(self.embed(inp))
        return emb


In [17]:
class TransformerConv(nn.Module):
    "Transformer model: https://arxiv.org/abs/1706.03762."
    def __init__(self, vocab_sz:int, n_layers:int, d_model:int, n_heads:int, d_head:int, 
                 **kwargs):
        super().__init__()
        self.embed = Embedder(vocab_sz, d_model)
        self.a1 = DownsampleLayer(d_model, n_heads, d_head)
        self.a2 = DownsampleLayer(d_model*2, n_heads, d_head)
        self.a3 = TransformerBlock(d_model*4, n_heads, d_head)
        self.a4 = UpsampleLayer(d_model*4, n_heads, d_head)
        self.a5 = UpsampleLayer(d_model*2, n_heads, d_head)
    
    def reset(self): pass
    
    def forward(self, x):
        inp = self.embed(x)
        x1 = self.a1(inp)
        x2 = self.a2(x1)
        x3 = self.a3(x2)
        x4 = self.a4(x3, x1)
        x5 = self.a5(x4, inp)
        return x5 #For the LinearDecoder

In [18]:
class TransformerBase(nn.Module):
    "Transformer model: https://arxiv.org/abs/1706.03762."
    def __init__(self, vocab_sz:int, n_layers:int, d_model:int, n_heads:int, d_head:int, 
                 embed_p:float=0.0, **kwargs):
        super().__init__()
        self.embed = Embedder(vocab_sz, d_model)
        self.layers = nn.ModuleList([TransformerBlock(d_model, n_heads, d_head) for k in range(n_layers)])
    
    def reset(self): pass
    
    def forward(self, x):
        inp = self.embed(x)
        for layer in self.layers: inp = layer(inp)
        return inp #For the LinearDecoder

In [19]:
vocab_sz = len(data.vocab.itos)
# config = tfmer_lm_config.copy(); config
config = {
    'n_layers': 5,
    'n_heads': 4,
    'd_model': 128,
    'd_head': 32,
    'embed_p': 0.0 # Embed p needs to be 0 to match bert-mask baseline
}

## Conv transformer

In [20]:
encoder = TransformerConv(vocab_sz, **config)
decoder = Decoder(vocab_sz, config['d_model'], tie_encoder=encoder.embed.embed)
model = nn.Sequential(encoder, decoder)
model.reset = lambda: True
model.apply(init_transformer)
learn = LanguageLearner(data, model, loss_func=CrossEntropyFlat(ignore_index=pad_idx))
learn.metrics = [bert_acc]
learn.callbacks = []
# learn.to_fp16();

In [21]:
learn.fit_one_cycle(2, 1e-3)

epoch,train_loss,valid_loss,bert_acc,time
0,3.584596,3.465516,0.452597,31:40
1,3.037114,2.943001,0.508547,31:43


## Baseline Fastai TXL

In [31]:
# from fastai.text.learner import get_language_model
# encoder = TransformerXL(vocab_sz, d_inner=config['d_model']*4, ctx_len=bptt, **config)
# decoder = LinearDecoder(vocab_sz, config['d_model'], output_p=0.0, tie_encoder=encoder.encoder)
# model = nn.Sequential(encoder, decoder)
# model.reset = lambda: True
# model.apply(init_transformer)

# learn = LanguageLearner(data, model, loss_func=CrossEntropyFlat(ignore_index=pad_idx))
# learn.metrics = [bert_acc]
# # learn.to_fp16();


In [32]:
# learn.fit_one_cycle(2, 1e-3)

## Baseline Transformer Base

In [20]:
encoder = TransformerBase(vocab_sz, **config)
decoder = Decoder(vocab_sz, config['d_model'], tie_encoder=encoder.embed.embed)
model = nn.Sequential(encoder, decoder)
model.reset = lambda: True
model.apply(init_transformer)
learn = LanguageLearner(data, model, loss_func=CrossEntropyFlat(ignore_index=pad_idx))
learn.callbacks = []
learn.metrics = [bert_acc]
# learn.to_fp16();

In [21]:
learn.fit_one_cycle(2, 1e-3)

epoch,train_loss,valid_loss,bert_acc,time
0,3.090801,2.985096,0.509065,33:11
1,2.774094,2.696457,0.541146,33:17


In [25]:
# learn.lr_find(num_it=300)
# learn.recorder.plot()